Vocabulary size = 30K

In [ ]:
class QuestionGeneration():
    def __init__(self, lm_gq, uqa_qg, regularization):
        self.lm_qg = lm_qg
        self.uqa_qg = uqa_qg
        self.regularization = regularization
        
    def generate_token(self, context: str, ans: str, list_question_tokens: list) -> str:
        '''
        Input:
            - context:
            - ans:
            - list_question_tokens: the history of generated question tokens.
            The form of list_question_tokens is a list of (q_token, class), where class is 0 for lm
            token and 1 for uqa token
        Output:
            - The next generated question token
        '''
        # contains the tokens of the generated question
        list_question_tokens = []
        # contains the classes of each token of the gen. question. Same len as list_question_tokens
        # 0 = LM, 1 = UQA
        list_token_classes = []
        qi = None
        
        while not self.finished_generation(qi):
            # Generate the probs of the ith query token using the lm and uqa models
            lm_qi_probs = self.lm_qg.forward(context, ans, list_question_tokens)
            uqa_qi_probs = self.uqa_qg.forward(context, ans, list_question_tokens)

            # Get the final token  prob distribution using regulatization
            qi_probs, qi_class = self.regularization(lm_qi_probs, uqa_qi_probs, 
                                                     list_question_tokens, list_token_classes)

            # Get the predicted token
            qi = torch.argmax(qi_probs, dim=0)
            list_question_tokens.append(VOCAB_IDX2_STR[qi].item())
            list_token_classes.append(qi_class)
            
            
    def finished_generation(self, question_token):
        return question_token == '?' or question_token =='EOS'

# Testing

## Vocabulary

In [9]:
import urllib.request
import numpy as np
word_url = "http://svnweb.freebsd.org/csrg/share/dict/words?view=co&content-type=text/plain"
response = urllib.request.urlopen(word_url)
long_txt = response.read().decode()
VOCAB_IDX2_STR = np.array(long_txt.splitlines()).reshape(-1,1)

In [10]:
VOCAB_IDX2_STR.shape

(25487, 1)

In [23]:
import torch
import torch.nn as nn

In [24]:
softmax = nn.Softmax(dim=0)

In [29]:
qi_probs = softmax(torch.empty(VOCAB_IDX2_STR.shape[0]).normal_(mean=4,std=0.5))

In [30]:
qi = torch.argmax(qi_probs, dim=0)

In [33]:
VOCAB_IDX2_STR[qi].item()

'Valparaiso'